In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available!")
    device = torch.device("cuda")
    print(f"using device: {device}")
else:
    print("gpu is not available, using cpu instead.")
    device = torch.device("cpu")

GPU is available!
using device: cuda


In [3]:
class ModelWrapper:
    def __init__(self, model_name):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Load model with half-precision if supported, or use device_map for efficient placement
        try:
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16 if torch.cuda.is_available() else None,
                device_map="auto"
            )
        except Exception as e:
            print(f"Error loading model: {e}")
            raise
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Enable gradient checkpointing for large models
        self.model.gradient_checkpointing_enable()

        # Debug: Check if model is on GPU
        print(f"Model loaded to device: {next(self.model.parameters()).device}")

    def generate_text(self, prompt, max_length=100, num_return_sequences=1):
        inputs = self.tokenizer(prompt, return_tensors="pt")
        inputs = {key: value.to(self.device) for key, value in inputs.items()}  # Move inputs to GPU
        outputs = self.model.generate(
            **inputs, max_length=max_length, num_return_sequences=num_return_sequences
        )
        generated_texts = [
            self.tokenizer.decode(output, skip_special_tokens=True) for output in outputs
        ]
        return generated_texts

In [4]:
# Example usage
if __name__ == "__main__":
    model_name = "AIDC-AI/Marco-o1"
    model_wrapper = ModelWrapper(model_name)

    prompt = "Once upon a time, in a land far, far away,"
    generated_texts = model_wrapper.generate_text(prompt, max_length=50, num_return_sequences=1)

    for i, text in enumerate(generated_texts):
        print(f"Generated Text {i+1}:\n{text}\n")


config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Model loaded to device: cuda:0
Generated Text 1:
Once upon a time, in a land far, far away, there was a wise old wizard who had a magic garden. In this garden, there were magical flowers that bloomed once every 7 days and magical trees that bore fruit once every 



In [8]:
prompt = "How many S's are there in Mississippi"
generated_texts = model_wrapper.generate_text(prompt, max_length=512, num_return_sequences=1)

for i, text in enumerate(generated_texts):
    print(f"Generated Text {i+1}:\n{text}\n")

Generated Text 1:
How many S's are there in Mississippi? To determine how many 'S's are in the word "Mississippi," let's analyze the word step by step.

First, let's write out the word:
M-I-S-S-I-S-S-I-P-P-I

Now, let's identify each letter and count the 'S's:
1. The first letter is M.
2. The second letter is I.
3. The third letter is S.
4. The fourth letter is S.
5. The fifth letter is I.
6. The sixth letter is S.
7. The seventh letter is S.
8. The eighth letter is I.
9. The ninth letter is P.
10. The tenth letter is P.
11. The eleventh letter is I.

From this analysis, we can see that the letters 'S' appear at positions 3, 4, 6, 7, and 11. That's a total of five 'S's.

To double-check, let's count them again:
- Position 3: S
- Position 4: S
- Position 6: S
- Position 7: S
- Position 11: S

Yes, there are indeed five 'S's in the word "Mississippi."

Therefore, the number of 'S's in Mississippi is \boxed{4}. Wait, that's incorrect based on the previous count. Let me recount carefully.


In [9]:
print(generated_texts)

['How many S\'s are there in Mississippi? To determine how many \'S\'s are in the word "Mississippi," let\'s analyze the word step by step.\n\nFirst, let\'s write out the word:\nM-I-S-S-I-S-S-I-P-P-I\n\nNow, let\'s identify each letter and count the \'S\'s:\n1. The first letter is M.\n2. The second letter is I.\n3. The third letter is S.\n4. The fourth letter is S.\n5. The fifth letter is I.\n6. The sixth letter is S.\n7. The seventh letter is S.\n8. The eighth letter is I.\n9. The ninth letter is P.\n10. The tenth letter is P.\n11. The eleventh letter is I.\n\nFrom this analysis, we can see that the letters \'S\' appear at positions 3, 4, 6, 7, and 11. That\'s a total of five \'S\'s.\n\nTo double-check, let\'s count them again:\n- Position 3: S\n- Position 4: S\n- Position 6: S\n- Position 7: S\n- Position 11: S\n\nYes, there are indeed five \'S\'s in the word "Mississippi."\n\nTherefore, the number of \'S\'s in Mississippi is \\boxed{4}. Wait, that\'s incorrect based on the previous 